# Final Project

Daryl Hurwitz, Joseph Dong, Jiaru

Installing all the libraries needed for Colab and have GPU enabled as Hardware accelerator.  The output for the hidden test will be the `prediction.mat` once all the cells are ran. 

In [ ]:
!pip install -U mne # THE MOST RECENT **STABLE** VERSION: 0.20.7
!pip install pytorch-lightning
!pip install torchtext
!pip install pytorch_model_summary
!pip install wandb

In [ ]:
import numpy as np
from scipy  import io, interpolate, ndimage, stats
from scipy.fftpack import fft, fftfreq
from scipy.signal import hamming, cwt, firwin, filtfilt, freqz, decimate, resample
from scipy.interpolate import interp1d
import os
import matplotlib.pyplot as plt
from pathlib import Path
import torch
from sklearn.preprocessing import MinMaxScaler, RobustScaler
import torch.nn.functional as F
import torch.optim.lr_scheduler as lr_scheduler
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.accelerators import find_usable_cuda_devices
from pytorch_lightning.callbacks import Callback, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from pytorch_model_summary import summary
import wandb
import shutil
import mne

## 0. Load in files

Upload `model1.ckpt`, `model2.ckpt`, `model3.ckpt`, and `truetest_data.mat` to colab.  Followed by running everything in the colab.  Ensure you're using GPU.

In [ ]:
# Pickle would not work....
Path(f"{Path().resolve()}/checkpoints").mkdir( exist_ok=True)
src1 = "/content/model1.ckpt"
src2 = "/content/model2.ckpt"
src3 = "/content/model3.ckpt"
dst = "/content/checkpoints"
shutil.move(src1, dst)
shutil.move(src2, dst)
shutil.move(src3, dst)
#ecog_data = io.loadmat('leaderboard_data.mat')
ecog_data = io.loadmat('truetest_data.mat')

## 1. Normalize

In [ ]:
def normalize(ecogdata):
    means = np.mean(ecogdata, axis=1, keepdims=True)
    stds = np.std(ecogdata, axis=1, keepdims=True)
    normalized_data = (ecogdata - means) / stds
    common_average = np.median(normalized_data, axis=0, keepdims=True)
    normalized_data = normalized_data - common_average
    return normalized_data

## 2. Bandpass

In [ ]:
def filtered(ecogdata, low_freq, high_freq, sample_rate):
    print("Removing Noise...")
    harmonics = np.array([i * 50 for i in range(1, (sample_rate // 2) // 50)])
    trans = ecogdata.T
    raw_filter = mne.filter.filter_data(trans, sfreq = sample_rate,
        l_freq = low_freq, h_freq = high_freq, filter_length='10s', phase = 'zero', 
        fir_window = 'hann', fir_design = 'firwin')
    
    print("Removing Harmonics...")
    raw_notched = mne.filter.notch_filter(raw_filter, Fs = sample_rate,
        freqs=harmonics, method='spectrum_fit', filter_length='10s')

    return raw_notched

## 3. Convolution with Morlet wavelet

In [ ]:
def morlet_conv(ecogs, low_freq, high_freq, sample_rate, n_wavelets, decimate = 1):
    data = ecogs
    num_channels = data.shape[0]
    convolutes =  mne.time_frequency.tfr_array_morlet(data.reshape(1, num_channels, -1), sfreq = sample_rate,
                                                        freqs = np.logspace(np.log10(low_freq), np.log10(high_freq), n_wavelets), decim = decimate,
                                                        output='power', verbose=10, n_jobs=6)[0]
    return convolutes

## 4. Downsample

In [ ]:
### Decimation is already carried out by MNE's wavelet library
def downsample_decimate(input, down_factor=10):
    """DEPRECATED, MNE's wavelet library already decimates
    input signal sampled at 1000Hz, output siganl sampled at 100Hz, so the downsampling factor is 10
    input shape (#samples, #wavelets, #channels), output shape (1/10 #samples, #wavelets, #channels)"""

    downsample = decimate(input, down_factor, axis=0, ftype='fir') # iir seems numerically unstable
    return downsample

## 5. RobustScalar

> Note: an alternative is to do Log10 on the wavelet spectra, then **do**/do not do scale_robust

In [ ]:

def scalerobust(ecogs_train, ecogs_val):
    num_channels = ecogs_train.shape[0]
    transformer = RobustScaler(unit_variance=True, quantile_range=(.1, .9))
    transformer.fit(ecogs_train.T.reshape(-1,num_wavelet*num_channels))
    ecog_data_scaled = transformer.transform(ecogs_train.T.reshape(-1,num_wavelet*num_channels)).reshape(-1,num_wavelet, num_channels).T
    ecog_data_val_scaled = transformer.transform(ecogs_val.T.reshape(-1,num_wavelet*num_channels)).reshape(-1,num_wavelet, num_channels).T

    return ecog_data_scaled, ecog_data_val_scaled
        

## 6. Signal Shift

In [ ]:
'''
The lag between the dataglove position measurement recording and the 
amplifier measurement is 37ms.  The relevant physiological range: 0 − 200ms.
'''

def signal_shift(ecogs, fingers, time : float, fs):
    """Shifts ecogs and finger signals by some set amount time. Requires fs"""

    n_samp_shift = int(time * fs)
    # the first motions do not depend on available data
    out_fing = fingers[..., n_samp_shift:] 
    # The latter spectrograms have no corresponding data
    out_ecog = ecogs[..., :ecogs.shape[2]-n_samp_shift]

    return out_ecog, out_fing


# Glove Preprocessing

## 7. Upsample with Bicubic interpolation 

In [ ]:
def finger_interpolation(dataglove_signal, super_sf, true_sf, desired_sf, interpType):
        '''
        Cubic interpolation since the data was sampled at 25Hz and then upsampled to 1000Hz
        '''
        downscale_ratio= super_sf // true_sf
        upscaling_ratio = desired_sf // true_sf
 
        dgT = dataglove_signal.T #(5, 300000)
        finger_flex_true_fs = dgT[:, ::downscale_ratio] #(super-sampled to 1 kHz) which is why we are down sampling (5, 7500)
        finger_flex_true_fs = np.c_[finger_flex_true_fs,finger_flex_true_fs.T[-1]]
        
        ts = np.asarray(range(finger_flex_true_fs.shape[1])) * upscaling_ratio

        interpolated_finger_flex_funcs = [interpolate.interp1d(ts, finger_flex_true_fs_ch, kind=interpType) for
                                finger_flex_true_fs_ch in finger_flex_true_fs]

        ts_needed_hz = np.asarray(range(finger_flex_true_fs.shape[1] * upscaling_ratio)[:-upscaling_ratio])  # Removing the extra added edge
        interpolated_finger_flex = np.array([[interpolated_finger_flex_func(t) for t in ts_needed_hz] for 
                                                interpolated_finger_flex_func in interpolated_finger_flex_funcs])

        return interpolated_finger_flex



## 8. MinMax Scaler 

> Note: when scaling finger features, I would only do affine scaling operations, since otherwise the finger data becomes distorted and our model has to learn on the distortion

In [ ]:
def scale_minmax(hands_train, hands_val):

    mms = MinMaxScaler().fit(hands_train.T)
    train = mms.transform(hands_train.T).T
    validate = mms.transform(hands_val.T).T
    
    return train, validate

## 9. Fit Model to ECoGs and Fingers

In [ ]:
# Hard Parameters
f_s_ecog = 1000 # Hz
f_s_dg = 25 # Hz
supered_f_s = 1000 # Hz
freq_low, freq_high = 40, 300 # Lower and upper filtration bounds
num_wavelet = 40 # Number of wavelets
delay = 0.2 # Time delay
downsampling_f_s = 100 # Hz
upsampling_f_s = downsampling_f_s # Hz
shift_time = 0.2 # s. Tunable parameter
split_train = 0.7 # proportion
subject_num = 1
Path(f"{Path().resolve()}/data").mkdir(parents=True, exist_ok=True)
PATH = f"{Path().resolve()}/data"
fs_ml = upsampling_f_s
n_window = round(0.2 * fs_ml)
n_stride = 1

### Run Preprocessing Pipeline

In [ ]:
def testpreprocess(ecogdata):
    
    # Preprocess ECoGs
    e = normalize(ecogdata)
    e = filtered(e, freq_low, freq_high, f_s_ecog)
    e = morlet_conv(e, freq_low, freq_high, f_s_ecog, num_wavelet, 10)
    
    # Robust scalar
    num_channels = e.shape[0]
    transformer = RobustScaler(unit_variance=True, quantile_range=(.1, .9))
    transformer.fit(e.T.reshape(-1,num_wavelet*num_channels))
    e_scaled = transformer.transform(e.T.reshape(-1,num_wavelet*num_channels)).reshape(-1, num_wavelet, num_channels).T

    
    # Time shift
    n_samp_shift = int(shift_time * upsampling_f_s)
    e_test = e_scaled[..., :e_scaled.shape[2]-n_samp_shift]
    print(e_test.shape)
    return e_test

### Save preprocessing output, Clear workspace, Reload preprocessing output

In [ ]:
def save_preprocessing(patient : int, ecogdata, dgdata = None,  path = PATH, val=None, reshape=False):

    """Saves the output of preprocess() into .npy arrays, to free up RAM
    
    Doesn't actually do the freeing up RAM part though"""
    num_channels = ecogdata.shape[2]
    Path(f"{path}/train").mkdir(parents=True, exist_ok=True)
    Path(f"{path}/val").mkdir(parents=True, exist_ok=True)
    Path(f"{path}/test").mkdir(parents=True, exist_ok=True)
    ecog_path = f"{path}/train/patient{patient}_ecog.npy" if val is None else f"{path}/val/patient{patient}_ecog.npy" if \
        val is True else f"{path}/test/patient{patient}_ecog.npy"
    dg_path = f"{path}/train/patient{patient}_hand.npy" if val is None else f"{path}/val/patient{patient}_hand.npy" if \
        val is True else f"{path}/test/patient{patient}_hand.npy"
    if reshape:
        ecog_data = ecogdata.reshape(num_channels*num_wavelet,-1)
    if os.path.isfile(ecog_path):
        os.remove(ecog_path)
    if os.path.isfile(dg_path):
        os.remove(dg_path)
    np.save(ecog_path, ecogdata)
    np.save(dg_path, dgdata)


def load_preprocessing(patient : int, path, val = None):
    """Loads preprocessing data from file. Same output format as preprocess()."""
    ecog_path = f"{path}/train/patient{patient}_ecog.npy" if val is None else f"{path}/val/patient{patient}_ecog.npy" if \
        val is True else f"{path}/test/patient{patient}_ecog.npy"
    dg_path = f"{path}/train/patient{patient}_hand.npy" if val is None else f"{path}/val/patient{patient}_hand.npy" if \
        val is True else f"{path}/test/patient{patient}_hand.npy"
    ecog_data = np.load(ecog_path)
    if val == False:
      dg_data = ecog_data
    else:
      dg_data = np.load(dg_path)
    return ecog_data, dg_data

### Define dataset classes

In [ ]:
class Dataset(Dataset):
    """Includes both an ECoG and its corresponding finger movements"""
    
    def __init__(self, ecog : str, dg : str, n_window : float, n_stride : int, train = False ):
        
        self.ecog = np.load(ecog).astype('float32')
        self.dg = np.load(dg).astype('float32')
        self.n_window = n_window
        self.n_stride = 1
        self.n_samples = self.ecog.shape[2]
        self.dataset_len = (self.n_samples - self.n_window) // self.n_stride       # Assuming all samples are the same length
        
        self.train = train # when to train, val, or test

    def __len__(self):
        
        return self.dataset_len
    
    def __getitem__(self, index):
        
        startind = self.n_stride * index
        stopind = startind + self.n_window

        return self.ecog[startind : stopind, ...], self.fing[startind : stopind, ...]



In [ ]:
class DataModule(pl.LightningDataModule):
    
    def __init__(self, patient : int, batch_size : 128, window_n : float,  path=PATH):
        """For reference, Fingerflex uses batch sizes of 128"""
        self.window_n = window_n
        
        self.patient = patient
        self.batch_size = batch_size
        self.path = path
    
    def setup(self, stage = None):

        if stage == "fit" or stage is None:
            self.ds_train = Dataset(f"{self.path}/train/patient{self.patient}_ecog.npy", 
                                    f"{self.path}/train/patient{self.patient}_hand.npy",
                                    n_window=self.window_n, Train = True )
            self.ds_val = Dataset(f"{self.path}/val/patient{self.patient}_ecog.npy", 
                                    f"{self.path}/val/patient{self.patient}_hand.npy",
                                    n_window=self.window_n)
        if stage == "test" or stage is None:
            self.ds_test = Dataset(f"{self.path}/test/patient{self.patient}_ecog.npy", 
                                   n_window=self.window_n )
        
    def train_dataloader(self):
        return DataLoader(self.ds_train, batch_size=self.batch_size, num_workers=3, shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.ds_val, batch_size=self.batch_size)
    
    def test_dataloader(self): 
        return DataLoader(self.ds_test, batch_size=self.batch_size)

def correlation_metric(x, y):
    cos_metric = nn.CosineSimilarity(dim=-1, eps=1e-08)
    cos_sim = torch.mean(cos_metric(x, y))
    return cos_sim

In [ ]:
class Runner(pl.LightningModule):
    
    def __init__(self, model, learning_rate = 8.42e-5, decay_rate = 1e-6):
        super().__init__()
        self.model = model 
        self.lr = learning_rate
        self.decay = decay_rate
        
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = F.mse_loss(y_hat, y)
        corr = correlation_metric(y_hat, y)
        self.log("Train_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)

        self.log(f"cosine_dst_train", corr, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return 0.5*loss + 0.5*(1. - corr) 
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = F.mse_loss(y_hat, y)
        corr = correlation_metric(y_hat, y)
        assert x.shape == y.shape  
        correlation = np.corrcoef(y_hat, y)[0, 1]
        self.log("Validation Loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log("Validation Correlation", correlation, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log("cosine_dst_val", corr, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        
        return y_hat 
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        assert x.shape == y.shape  
        correlation = np.corrcoef(y_hat, y)[0, 1]
        loss = F.mse_loss(y_hat, y)
        self.log("Validation Correlation", correlation, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log("test_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr, weight_decay=1e-6) # set optimizer, lr and L2 regularization coeff
        scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=3, verbose=True)  # Add ReduceLROnPlateau scheduler
        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': scheduler,
                'monitor': 'val_loss'
            }
        }

# Model

In [ ]:
class Conv(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, stride=1, dilation=1, prob=0.1):
        super(Conv, self).__init__()
        
        self.conv1d = nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size, bias=False, padding='same')
        self.norm = nn.LayerNorm(out_channels)
        self.activation = nn.GELU()  ## maybe swap this out with other activation function
        self.drop = nn.Dropout(p=prob)
        self.downsample = nn.MaxPool1d(kernel_size=stride, stride=stride)
        self.stride = stride
        self.in_channels = in_channels
        self.out_channels = out_channels
        
        
    def forward(self, x):
        
        x = self.conv1d(x)
        x = torch.transpose(x, -2, -1) 
        x = self.norm(x)
        x = torch.transpose(x, -2, -1)
        x = self.activation(x)
        x = self.drop(x)
        x = self.downsample(x)

        return x

    
    
class UpConv(nn.Module):

    def __init__(self, scale, **args):
        super(UpConv, self).__init__()
        self.conv_block = Conv(**args)
        self.upsample = nn.Upsample(scale_factor=scale, mode='linear', align_corners=False)

            
    def forward(self, x):
        
        x = self.conv_block(x)
        x = self.upsample(x)
        return x    
    

class EDModel_Skip(nn.Module):

    def __init__(self, n_electrodes=30, n_freqs = 16, n_channels_out=21,  channels = [8, 16, 32, 32],  
                 kernel_sizes=[3, 3, 3], strides=[4, 4, 4], dilation=[1, 1, 1] ):
        
        super(EDModel_Skip, self).__init__()
        self.n_electrodes = n_electrodes
        self.n_freqs = n_freqs
        self.in_features = n_freqs * n_electrodes
        self.n_channels_out = n_channels_out
        self.depth = len(channels)-1
        self.spatial_reduce = Conv(self.in_features, channels[0], kernel_size=3) 
        

        self.downsample_blocks = nn.ModuleList([Conv(channels[i], channels[i+1], kernel_sizes[i],stride=strides[i],
                                                     dilation=dilation[i]) for i in range(self.depth)])
        channels = [ch for ch in channels[:-1]] + channels[-1:] # rearranges the channels to get ready for outpout
        self.upsample_blocks = nn.ModuleList([UpConv(scale=strides[i], in_channels=channels[i+1] if i == self.depth-1 else channels[i+1]*2 ,
                                                     out_channels=channels[i], kernel_size=kernel_sizes[i]) for i in range(self.depth-1, -1, -1)])
        self.conv1x1_one = nn.Conv1d(channels[0]*2, self.n_channels_out, kernel_size=1, padding='same') 
      
    def forward(self, x):

        batch, elec, n_freq, time = x.shape
        x = x.reshape(batch, -1, time)  
        x = self.spatial_reduce(x)
        skip_connection = []
        
        for i in range(self.depth):
            skip_connection.append(x)
            x = self.downsample_blocks[i](x)

        for i in range(self.depth):
            x = self.upsample_blocks[i](x)
            x = torch.cat((x, skip_connection[-1 - i]), dim=1)
        
        x = self.conv1x1_one(x)

        return x

In [ ]:
class TestCallback:
    def __init__(self, test_x, subject_num, fg_num):
        super().__init__()
        self.test_x = test_x.T
        self.fg_num = fg_num
        self.subject = subject_num

    def test(self, pl_module):
        with torch.no_grad():
            size = 64
            bound = self.test_x.shape[0]//size * size
            X_test = self.test_x[:bound]
            x_batch = torch.from_numpy(X_test).float().to("cuda:0")
            x_batch = x_batch.T
            x_batch = torch.unsqueeze(x_batch, 0)
            y_hat = pl_module.model(x_batch)[0]
            y_hat = y_hat.cpu().detach().numpy()
            stride = 1
            y_prediction = y_hat.T[::int(stride*(downsampling_f_s/100)), :]
            y_prediction = np.pad(y_prediction, ((15, 15), (0, 0)), mode='edge')
            y_prediction_tensor = torch.tensor(y_prediction).float()
            y_prediction_upsampled = F.interpolate(y_prediction_tensor.unsqueeze(0).unsqueeze(0), scale_factor=(10, 1), mode="nearest")
            y_prediction_upsampled = y_prediction_upsampled.squeeze(0).squeeze(0).numpy()
            y_prediction_upsampled = ndimage.gaussian_filter1d(y_prediction_upsampled.T, sigma=1).T

            np.save(f"{Path().resolve()}/submit/prediction{self.subject}.npy", y_prediction_upsampled)

# Train/Test

In [ ]:
window = 256 
learn_r = 8.42e-5 # learning rate
decay_r = 1e-6 # Decay rate
batchsize = 128

In [ ]:
for i in range(3):
    subject = i + 1
    
    encoder_args = dict(channels = [32, 32, 64, 64, 128, 128, 256], 
                        kernel_sizes=[ 9, 7, 7, 5, 5, 5],
                        strides=[2, 2, 2, 2, 2, 2],
                        dilation=[1,  1, 1, 1, 1,  1],
                        n_electrodes = num_channels,
                        n_freqs = num_wavelets,
                        n_channels_out = num_fingers) # A set of features for the model

    model = EDModel_Skip(**encoder_args).to("cuda:0")
    lighning_wrapper = Runner(model) # Wrapping in pytorch-lightning class



    dm = DataModule(subject, batchsize, window, PATH)

In [ ]:
Path(f"{Path().resolve()}/submit").mkdir(parents=True, exist_ok=True)
for i in range(3):
    subject = i + 1
    #ecog_test = testpreprocess(ecog_data['leaderboard_ecog'][i][0])
    #print(ecog_test.shape[0])
    ecog_test = testpreprocess(ecog_data['truetest_data'][i][0])
    num_channels = ecog_test.shape[0]
    num_wavelets = ecog_test.shape[1]
    num_fingers = 5
    encoder_args = dict(channels = [32, 32, 64, 64, 128, 128, 256], 
                        kernel_sizes=[9, 7, 7, 5, 5, 5],
                        strides=[2, 2, 2, 2, 2, 2],
                        dilation=[1, 1, 1, 1, 1, 1],
                        n_electrodes = num_channels,
                        n_freqs = num_wavelets,
                        n_channels_out = num_fingers) 
    trained_model = Runner.load_from_checkpoint(checkpoint_path = f"{Path().resolve()}/checkpoints/model{subject}.ckpt", model = EDModel_Skip(**encoder_args))
    test_callback = TestCallback(ecog_test, subject, num_fingers)
    trained_model = trained_model.to("cuda:0")
    test_callback.test(trained_model) 

In [30]:
predictions = np.zeros((3,1), dtype=object)
predictions[0,0] = np.load(f"{Path().resolve()}/submit/prediction1.npy")
predictions[1,0] = np.load(f"{Path().resolve()}/submit/prediction2.npy")
predictions[2,0] = np.load(f"{Path().resolve()}/submit/prediction3.npy")
#save the array using the right format
io.savemat('predictions.mat', {'predicted_dg':predictions})
     